In [1]:
import pandas as pd
import os
import glob
import boto3
import toml

In [2]:
df_time_profile = pd.read_csv("../data/Time Profile for BOT.csv")

In [3]:
df_time_profile.head()

,Start Date,End Date,Day,Fiscal Month,Fiscal Quarter,Fiscal Year
0,1/27/2014,1/27/2014,1/27/2014,Feb-FY15,FY15-Q1,FY15
1,1/28/2014,1/28/2014,1/28/2014,Feb-FY15,FY15-Q1,FY15
2,1/29/2014,1/29/2014,1/29/2014,Feb-FY15,FY15-Q1,FY15
3,1/30/2014,1/30/2014,1/30/2014,Feb-FY15,FY15-Q1,FY15
4,1/31/2014,1/31/2014,1/31/2014,Feb-FY15,FY15-Q1,FY15


In [4]:
with open('../.streamlit/secrets.toml', 'r') as f:
    credentials = toml.load(f)

In [5]:
aws_access_key_id = credentials['DATABASE']['AWS_ACCESS_KEY_ID']
aws_secret_access_key = credentials['DATABASE']['AWS_SECRET_ACCESS_KEY']
region_name = credentials['DATABASE']['REGION_NAME']

bucket_name = credentials['DATABASE']['BUCKET']
multiplier = credentials['DATABASE']['NUMBER']

In [6]:
data_path = "../../customer_allocation_data"

csv_files = glob.glob(os.path.join(data_path, "*.csv"))

# Initialize S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

In [7]:
# df_subset = df_time_profile[df_time_profile['Fiscal Quarter'] == 'FY25-Q4'].copy()

In [8]:
df_time_profile['Day'] = pd.to_datetime(df_time_profile['Day'])
df_time_profile['Weekly Dates'] = df_time_profile['Day'].apply(lambda x: x + pd.Timedelta(days=(6 - x.weekday())) if x.weekday() <= 6 else None)
df_time_profile['Weekly Dates'] = df_time_profile['Weekly Dates'].apply(lambda x: f"W/e {x.strftime('%d %b %y')}" if x else None)

In [9]:
df_time_profile.head()

,Start Date,End Date,Day,Fiscal Month,Fiscal Quarter,Fiscal Year,Weekly Dates
0,1/27/2014,1/27/2014,2014-01-27,Feb-FY15,FY15-Q1,FY15,W/e 02 Feb 14
1,1/28/2014,1/28/2014,2014-01-28,Feb-FY15,FY15-Q1,FY15,W/e 02 Feb 14
2,1/29/2014,1/29/2014,2014-01-29,Feb-FY15,FY15-Q1,FY15,W/e 02 Feb 14
3,1/30/2014,1/30/2014,2014-01-30,Feb-FY15,FY15-Q1,FY15,W/e 02 Feb 14
4,1/31/2014,1/31/2014,2014-01-31,Feb-FY15,FY15-Q1,FY15,W/e 02 Feb 14


In [ ]:
from datetime import date
current_date = date.today()
current_date_pandas = pd.to_datetime(current_date)
current_fiscal_quarter = df_time_profile[df_time_profile['Day'] == current_date_pandas]['Fiscal Quarter'].values[0]

In [ ]:
weekly_dates = list(df_time_profile[df_time_profile['Fiscal Quarter'] == current_fiscal_quarter]['Weekly Dates'].unique()) + [current_fiscal_quarter]

In [17]:
import pandas as pd

# Input: List of unique values for columns
unique_values = weekly_dates

# Define the primary level of the index
index_primary = ['BUF', 'JFF', 'Demand', 'Shipped', 'Allocation']

# Create a MultiIndex with the original levels and the 'Total' group
index = pd.MultiIndex.from_product([index_primary, ['Total']])

# Create the DataFrame with the MultiIndex and columns, filled with zeros
df_result = pd.DataFrame(0, index=index, columns=unique_values)


In [18]:
df_result.head()

,,W/e 03 Nov 24,W/e 10 Nov 24,W/e 17 Nov 24,W/e 24 Nov 24,W/e 01 Dec 24,W/e 08 Dec 24,W/e 15 Dec 24,W/e 22 Dec 24,W/e 29 Dec 24,W/e 05 Jan 25,W/e 12 Jan 25,W/e 19 Jan 25,W/e 26 Jan 25,FY25-Q4
BUF,Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0
JFF,Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Demand,Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Shipped,Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Allocation,Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
date_columns = ['Ord Date', 'CRD Date Adjusted', 'Inv Date', 'Pl. GI Date', 'MAD', 'Allocation Date']

In [20]:
import pandas as pd
from datetime import datetime

def adjust_crd_date(filename: str, dataframe):
    try:
        # Assuming filename starts with 'CustAllocDownload' and the date is the next 8 characters
        date_str_index = filename.index("CustAllocDownload") + len("CustAllocDownload")
        file_date_str = filename[date_str_index:date_str_index + 8]
        
        # Parse the extracted date and format it to 'YYYY-MM-DD'
        file_date = datetime.strptime(file_date_str, '%Y%m%d')
        file_date_formatted = file_date.strftime('%Y-%m-%d')
    except (ValueError, IndexError):
        # Handle cases where the filename does not contain a valid date
        raise ValueError(f"Filename '{filename}' does not contain a valid date in the expected format 'CustAllocDownloadYYYYMMDD'.")

    # Convert 'CRD Date' column to datetime format for comparison, handling errors
    dataframe['CRD Date'] = pd.to_datetime(dataframe['CRD Date'], format='%Y-%m-%d', errors='coerce')

    # Adjust 'CRD Date' based on the opposite comparison with file date
    dataframe['CRD Date Adjusted'] = dataframe['CRD Date'].apply(
        lambda x: file_date_formatted if pd.notnull(x) and x < file_date else (x.strftime('%Y-%m-%d') if pd.notnull(x) else pd.NaT)
    )

    return dataframe


In [21]:
# def map_calendar_to_fiscal(data_time_profile, data):
    
#     # Convert Day in df_time_profile to datetime
#     data_time_profile['Day'] = pd.to_datetime(data_time_profile['Day'], format='%m/%d/%Y')

#     # Create a mapping dictionary to associate each date with fiscal attributes
#     fiscal_mapping = data_time_profile.set_index('Day')[['Fiscal Month', 'Fiscal Quarter', 'Fiscal Year']]

#     # Convert all date columns in df to datetime
#     date_columns = ['Ord Date', 'CRD Date Adjusted', 'Inv Date', 'Pl. GI Date', 'MAD', 'Allocation Date']
#     for col in date_columns:
#         data[col] = pd.to_datetime(data[col], format='%Y-%m-%d')

#     # For each date column in df, map fiscal attributes and create new columns
#     for col in date_columns:
#         data[f'{col} Fiscal Month'] = data[col].map(fiscal_mapping['Fiscal Month'])
#         data[f'{col} Fiscal Quarter'] = data[col].map(fiscal_mapping['Fiscal Quarter'])
#         data[f'{col} Fiscal Year'] = data[col].map(fiscal_mapping['Fiscal Year'])

#     return data

In [22]:
def map_calendar_to_fiscal(data_time_profile, data):
    data_time_profile['Day'] = pd.to_datetime(data_time_profile['Day'], format='%m/%d/%Y')
    data_time_profile['Weekly Dates'] = data_time_profile['Day'].apply(
        lambda x: x + pd.Timedelta(days=(6 - x.weekday()))
    )
    data_time_profile['Weekly Dates'] = data_time_profile['Weekly Dates'].apply(
        lambda x: f"W/e {x.strftime('%d %b %y')}"
    )
    fiscal_mapping = data_time_profile.set_index('Day')[['Fiscal Month', 'Fiscal Quarter', 'Fiscal Year', 'Weekly Dates']]
    date_columns = ['Ord Date', 'CRD Date', 'CRD Date Adjusted', 'Inv Date', 'Pl. GI Date', 'MAD', 'Allocation Date']
    for col in date_columns:
        data[col] = pd.to_datetime(data[col], format='%Y-%m-%d')

    for col in date_columns:
        data[f'{col} Fiscal Month'] = data[col].map(fiscal_mapping['Fiscal Month'])
        data[f'{col} Fiscal Quarter'] = data[col].map(fiscal_mapping['Fiscal Quarter'])
        data[f'{col} Fiscal Year'] = data[col].map(fiscal_mapping['Fiscal Year'])
        data[f'{col} Weekly Dates'] = data[col].map(fiscal_mapping['Weekly Dates'])

    return data

In [23]:
# This is used to upload the masked customer allocation download files in Amazon S3
for i, csv_file in enumerate(csv_files):
    df = pd.read_csv(csv_file, encoding='ISO-8859-1')
    numerical_features = df.select_dtypes(include=['int64', 'float64']).columns
    df[numerical_features] = df[numerical_features] * multiplier

    df_adjusted = adjust_crd_date(
        filename=csv_file.split("/")[-1],
        dataframe=df
    )
    df_date_converted = map_calendar_to_fiscal(
        data_time_profile=df_time_profile, 
        data=df_adjusted
    )

    # individual_file = csv_file.split("/")[-1]
    # save_path = f"../../customer_allocation_data_masked/{individual_file}"
    # df_date_converted.to_csv(save_path, index=False)

/tmp/ipykernel_96798/3264791628.py:3: DtypeWarning: Columns (1,3,5,6,7,9,10,14,18,20,21,22,23,27,28,30,31,33,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, encoding='ISO-8859-1')
/tmp/ipykernel_96798/3264791628.py:3: DtypeWarning: Columns (1,3,5,6,7,9,10,14,18,20,21,22,23,27,28,30,31,33,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, encoding='ISO-8859-1')
/tmp/ipykernel_96798/3264791628.py:3: DtypeWarning: Columns (1,3,5,6,7,9,10,14,18,20,21,22,23,27,28,30,31,33,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, encoding='ISO-8859-1')
/tmp/ipykernel_96798/3264791628.py:3: DtypeWarning: Columns (1,3,5,6,7,9,10,14,18,20,21,22,23,27,28,30,31,33,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, encoding='ISO-8859-1')
/tmp/ipykernel_96798/3264791628.py:

In [24]:
df_date_converted.head()

,Demand Type,Ord Type,SKU,Part #,Adaptor,Customer #,Cust Name,Region,SalesPriority,End Customer,...,Pl. GI Date Fiscal Year,Pl. GI Date Weekly Dates,MAD Fiscal Month,MAD Fiscal Quarter,MAD Fiscal Year,MAD Weekly Dates,Allocation Date Fiscal Month,Allocation Date Fiscal Quarter,Allocation Date Fiscal Year,Allocation Date Weekly Dates
0,Allocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Jul-FY25,FY25-Q2,FY25,W/e 21 Jul 24
1,Allocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Aug-FY25,FY25-Q3,FY25,W/e 04 Aug 24
2,Allocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Sep-FY25,FY25-Q3,FY25,W/e 15 Sep 24
3,Allocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,May-FY25,FY25-Q2,FY25,W/e 05 May 24
4,Allocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Jul-FY25,FY25-Q2,FY25,W/e 21 Jul 24


In [25]:
df_date_converted.columns

Index(['Demand Type', 'Ord Type', 'SKU', 'Part #', 'Adaptor', 'Customer #',
       'Cust Name', 'Region', 'SalesPriority', 'End Customer', 'End Cust Name',
       'Sale Ord #', 'Line #', 'Sch Ln#', 'PO #', 'Quantity', 'Unit Price',
       'Amount', 'Ord Date', 'CRD Date', 'Inv Date', 'Pl. GI Date', 'MAD',
       'Ship Plnt', 'Business Unit', 'Types', 'Family', 'Line',
       'Higher Level Item', 'Del #', 'Del Date', 'Rejection', 'Allocation Qty',
       'Allocation Date', 'Planning Group', 'Alt Allocation',
       'Product End Customer', 'Product End Customer Name', 'Planner Profile',
       'Planning Level', 'CRD Date Adjusted', 'Ord Date Fiscal Month',
       'Ord Date Fiscal Quarter', 'Ord Date Fiscal Year',
       'Ord Date Weekly Dates', 'CRD Date Fiscal Month',
       'CRD Date Fiscal Quarter', 'CRD Date Fiscal Year',
       'CRD Date Weekly Dates', 'CRD Date Adjusted Fiscal Month',
       'CRD Date Adjusted Fiscal Quarter', 'CRD Date Adjusted Fiscal Year',
       'CRD Date Adjus

In [37]:
df_time_profile

,Start Date,End Date,Day,Fiscal Month,Fiscal Quarter,Fiscal Year,Weekly Dates
0,1/27/2014,1/27/2014,2014-01-27,Feb-FY15,FY15-Q1,FY15,W/e 02 Feb 14
1,1/28/2014,1/28/2014,2014-01-28,Feb-FY15,FY15-Q1,FY15,W/e 02 Feb 14
2,1/29/2014,1/29/2014,2014-01-29,Feb-FY15,FY15-Q1,FY15,W/e 02 Feb 14
3,1/30/2014,1/30/2014,2014-01-30,Feb-FY15,FY15-Q1,FY15,W/e 02 Feb 14
4,1/31/2014,1/31/2014,2014-01-31,Feb-FY15,FY15-Q1,FY15,W/e 02 Feb 14
...,...,...,...,...,...,...,...
6568,1/21/2032,1/21/2032,1932-01-21,Jan-FY32,FY32-Q4,FY32,W/e 24 Jan 32
6569,1/22/2032,1/22/2032,1932-01-22,Jan-FY32,FY32-Q4,FY32,W/e 24 Jan 32
6570,1/23/2032,1/23/2032,1932-01-23,Jan-FY32,FY32-Q4,FY32,W/e 24 Jan 32
6571,1/24/2032,1/24/2032,1932-01-24,Jan-FY32,FY32-Q4,FY32,W/e 24 Jan 32


In [26]:
def index_time_profile(df_time_profile, current_fiscal_quarter):

    unique_values = list(df_time_profile[df_time_profile['Fiscal Quarter'] == current_fiscal_quarter]['Weekly Dates'].unique()) + [current_fiscal_quarter]
    index_primary = ['BUF', 'JFF', 'Demand', 'Shipped', 'Supply', 'Allocation']
    index = pd.MultiIndex.from_product([index_primary, ['Total']])
    df_result = pd.DataFrame(0, index=index, columns=unique_values)

    return df_result

In [27]:
df_result = index_time_profile(
    df_time_profile=df_time_profile,
    current_fiscal_quarter=current_fiscal_quarter
)

In [28]:
df_result.head()

,,W/e 03 Nov 24,W/e 10 Nov 24,W/e 17 Nov 24,W/e 24 Nov 24,W/e 01 Dec 24,W/e 08 Dec 24,W/e 15 Dec 24,W/e 22 Dec 24,W/e 29 Dec 24,W/e 05 Jan 25,W/e 12 Jan 25,W/e 19 Jan 25,W/e 26 Jan 25,FY25-Q4
BUF,Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0
JFF,Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Demand,Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Shipped,Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Supply,Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
# Apply display format
pd.options.display.float_format = '{:.0f}'.format

In [33]:
df_date_converted.loc[df_date_converted['Demand Type'] == 'PULLED', 'Demand Type'] = 'SHIPPED'
df_date_converted.loc[df_date_converted['Demand Type'] == 'ZSCH', 'Demand Type'] = 'BOOKED'

In [ ]:
def convert_to_anaplan(df_date_converted, df_result, current_fiscal_quarter):
    
    for demand_type in df_date_converted['Demand Type'].unique():
        if demand_type == 'Allocation':
            df_demand_type = df_date_converted[df_date_converted['Demand Type'] == demand_type].copy()
            df_fiscal_date = df_demand_type[df_demand_type['Allocation Date Fiscal Quarter'] == current_fiscal_quarter].copy()
            grouped_sum = df_fiscal_date.groupby(by=['Allocation Date Weekly Dates'])['Allocation Qty'].sum()
            for weekly_date, qty in grouped_sum.items():
                if weekly_date in df_result.columns:
                    df_result.loc[('Allocation', 'Total'), weekly_date] = qty

        elif demand_type == 'BUF':
            df_demand_type = df_date_converted[df_date_converted['Demand Type'] == demand_type].copy()
            df_fiscal_date = df_demand_type[df_demand_type['CRD Date Fiscal Quarter'] == current_fiscal_quarter].copy()
            grouped_sum = df_fiscal_date.groupby(by=['CRD Date Weekly Dates'])['Quantity'].sum()
            for weekly_date, qty in grouped_sum.items():
                if weekly_date in df_result.columns:
                    df_result.loc[('BUF', 'Total'), weekly_date] = qty

        elif demand_type == 'JFF':
            df_demand_type = df_date_converted[df_date_converted['Demand Type'] == demand_type].copy()
            df_fiscal_date = df_demand_type[df_demand_type['CRD Date Fiscal Quarter'] == current_fiscal_quarter].copy()
            grouped_sum = df_fiscal_date.groupby(by=['CRD Date Weekly Dates'])['Quantity'].sum()
            for weekly_date, qty in grouped_sum.items():
                if weekly_date in df_result.columns:
                    df_result.loc[('JFF', 'Total'), weekly_date] = qty

        elif demand_type == 'Supply':
            df_demand_type = df_date_converted[df_date_converted['Demand Type'] == demand_type].copy()
            df_fiscal_date = df_demand_type[df_demand_type['CRD Date Fiscal Quarter'] == current_fiscal_quarter].copy()
            grouped_sum = df_fiscal_date.groupby(by=['CRD Date Weekly Dates'])['Quantity'].sum()
            for weekly_date, qty in grouped_sum.items():
                if weekly_date in df_result.columns:
                    df_result.loc[('Supply', 'Total'), weekly_date] = qty

        elif demand_type == 'SHIPPED':
            df_demand_type = df_date_converted[df_date_converted['Demand Type'] == demand_type].copy()
            df_fiscal_date = df_demand_type[df_demand_type['Pl. GI Date Fiscal Quarter'] == current_fiscal_quarter].copy()
            grouped_sum = df_fiscal_date.groupby(by=['Pl. GI Date Weekly Dates'])['Quantity'].sum()
            for weekly_date, qty in grouped_sum.items():
                if weekly_date in df_result.columns:
                    df_result.loc[('Shipped', 'Total'), weekly_date] = qty

        elif demand_type == 'BOOKED':
            df_demand_type = df_date_converted[df_date_converted['Demand Type'] == demand_type].copy()
            df_fiscal_date = df_demand_type[df_demand_type['CRD Date Adjusted Fiscal Quarter'] == current_fiscal_quarter].copy()
            grouped_sum = df_fiscal_date.groupby(by=['CRD Date Adjusted Weekly Dates'])['Quantity'].sum()
            for weekly_date, qty in grouped_sum.items():
                if weekly_date in df_result.columns:
                    df_result.loc[('Demand', 'Total'), weekly_date] = qty

    for weekly_date in df_result.columns:
        if all([weekly_date in df_result.columns]):
            df_result.loc[('Allocation - Demand Delta', 'Total'), weekly_date] = (
                df_result.loc[('Allocation', 'Total'), weekly_date]
                - df_result.loc[('Demand', 'Total'), weekly_date]
            )
            df_result.loc[('Supply - Demand Delta', 'Total'), weekly_date] = (
                df_result.loc[('Supply', 'Total'), weekly_date]
                - df_result.loc[('Demand', 'Total'), weekly_date]
            )

            df_result.loc[('Supply - Allocation Delta', 'Total'), weekly_date] = (
                df_result.loc[('Supply', 'Total'), weekly_date]
                - df_result.loc[('Allocation', 'Total'), weekly_date]
            )


    final_column = df_result.columns[-1]

    df_result.drop([final_column], axis=1, inplace=True)
    final_column_list = final_column.split('-')
    renamed_column = final_column_list[1] + " " + final_column_list[0]
    df_result[renamed_column] = df_result.sum(axis=1)

    output_df = df_result.copy()

    return output_df


In [31]:
df_date_converted.head()

,Demand Type,Ord Type,SKU,Part #,Adaptor,Customer #,Cust Name,Region,SalesPriority,End Customer,...,Pl. GI Date Fiscal Year,Pl. GI Date Weekly Dates,MAD Fiscal Month,MAD Fiscal Quarter,MAD Fiscal Year,MAD Weekly Dates,Allocation Date Fiscal Month,Allocation Date Fiscal Quarter,Allocation Date Fiscal Year,Allocation Date Weekly Dates
0,Allocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Jul-FY25,FY25-Q2,FY25,W/e 21 Jul 24
1,Allocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Aug-FY25,FY25-Q3,FY25,W/e 04 Aug 24
2,Allocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Sep-FY25,FY25-Q3,FY25,W/e 15 Sep 24
3,Allocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,May-FY25,FY25-Q2,FY25,W/e 05 May 24
4,Allocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Jul-FY25,FY25-Q2,FY25,W/e 21 Jul 24


In [34]:
df_date_converted['Demand Type'].unique()

array(['Allocation', 'Supply', 'Locked Allocation', 'JFF', 'BOOKED',
       'SHIPPED', 'BUF', 'RSF'], dtype=object)

In [35]:
final_output = convert_to_anaplan(
    df_date_converted=df_date_converted,
    df_result=df_result,
    current_fiscal_quarter=current_fiscal_quarter
)

/tmp/ipykernel_96798/1602299486.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '910156.104060732' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_result.loc[('Allocation', 'Total'), weekly_date] = qty
/tmp/ipykernel_96798/1602299486.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6433711.415006826' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_result.loc[('Allocation', 'Total'), weekly_date] = qty
/tmp/ipykernel_96798/1602299486.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '693954.2740977679' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_result.loc[('Allocation', 'Total'), weekly_date] = qty

In [36]:
final_output

,,W/e 03 Nov 24,W/e 10 Nov 24,W/e 17 Nov 24,W/e 24 Nov 24,W/e 01 Dec 24,W/e 08 Dec 24,W/e 15 Dec 24,W/e 22 Dec 24,W/e 29 Dec 24,W/e 05 Jan 25,W/e 12 Jan 25,W/e 19 Jan 25,W/e 26 Jan 25,Q4 FY25
BUF,Total,0,0,0,177705127,0,0,0,162550815,0,0,0,0,88823645,429079587
JFF,Total,0,0,0,155220281,0,0,0,209235098,0,0,0,0,199300867,563756247
Demand,Total,23000007,22683611,19440561,21027379,22559919,23690848,21610219,26402245,23274046,18055310,25462397,20202761,20594764,288004068
Shipped,Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Supply,Total,48729326,49580800,49113234,49022903,36808701,36009381,35186764,35411629,28943011,28321172,28518247,28861054,28527732,483033954
Allocation,Total,6433711,1702448,1558784,20164062,910156,662617,242602,28457341,682756,693954,543863,396285,27539067,89987645
Allocation - Demand Delta,Total,-16566296,-20981163,-17881777,-863317,-21649763,-23028230,-21367618,2055096,-22591290,-17361356,-24918534,-19806476,6944302,-198016423
Supply - Demand Delta,Total,25729319,26897189,29672673,27995524,14248781,12318534,13576545,9009384,5668965,10265862,3055850,8658293,7932968,195029886
Supply - Allocation Delta,Total,42295614,47878352,47554450,28858841,35898545,35346764,34944163,6954288,28260255,27627218,27974384,28464769,988666,393046309
